In [ ]:
# pip install flask python-dotenv langchain openai

In [1]:
import os
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.utilities import GoogleSerperAPIWrapper
from langchain.agents import AgentExecutor, create_structured_chat_agent, Tool
from langchain.prompts import PromptTemplate

In [2]:
load_dotenv()

True

In [3]:
class Event:

    def __init__(self, name, date, location, description):
        self.name = name
        self.date = date
        self.location = location
        self.description = description

    def __str__(self):
        return f"{self.name} on {self.date} at {self.location} - {self.description}"

In [4]:
#tools

def add_to_calender(event):
    print(f"Added event: {event.name} to calender")
    
def web_search(query):
    if query:
        return GoogleSerperAPIWrapper().search(query)
    else:
        return "Please provide a search query"

def create_event(name, date, location, description):
    if name and date and location and description:
        return Event(name, date, location, description)
    else:
        return "Please provide all the details"

def ask_user_interest_in_event(event):
    print(f"Are you interested in {event}?")
    response = input()
    return response

def ask_user_for_event_type():
    print("What type of event are you looking for?")
    response = input()
    return response

tools = [
    Tool(name="add_to_calender", func=add_to_calender, description="Adds an event to users calender"),
    Tool(name="web_search", func=web_search, description="Searches the web for a query"),
    Tool(name="create_event", func=create_event, description="Creates an event")
]

In [5]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.3)

/var/folders/2j/h6y_zlpd5sq1fhr8v9460g600000gn/T/ipykernel_39329/2085404453.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.3)


In [6]:
prompt_template = PromptTemplate(
    input_variables=["tool_names", "agent_scratchpad", "tools"],
    template=(
        "You are an intelligent assistant that helps users find and manage events. "
        "You have access to the following tools: {tool_names}.\n\n"
        "Here are the detailed descriptions of the tools:\n{tools}\n\n"
        "Your task is to ask the user what type of event they are interested in, "
        "search the web for events that match their query, and present those events. "
        "For each event, ask if the user is interested. If they are, create an event "
        "and add it to their calendar.\n\n"
        "Use the tools efficiently to fulfill the user's request.\n\n"
        "{agent_scratchpad}"
    ),
)


In [7]:
agent = create_structured_chat_agent(
    tools=tools,
    llm=llm,
    prompt=prompt_template
    )

In [8]:
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [9]:
agent_executor.invoke({"input": "Search events in New York", "agent_scratchpad": ""})

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: Hello! How can I assist you today? Are you looking for a specific type of event or activity?
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE